### Intro Gym & Apprentissage par renforcement.

Installer le package Gymnasium

In [1]:
 !pip install gymnasium

   ---------------------------------------- 0.0/953.9 kB ? eta -:--:--
   --------------------------------------  952.3/953.9 kB 30.4 MB/s eta 0:00:01
   --------------------------------------  952.3/953.9 kB 30.4 MB/s eta 0:00:01
   ---------------------------------------- 953.9/953.9 kB 7.5 MB/s eta 0:00:00


DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Import des package nécessaires

In [1]:
import numpy as np
import gymnasium as gym
import random

#### Introduction
[Gym](https://gymnasium.farama.org/content/basic_usage/) est une boîte à outils pour le développement et la comparaison d'algorithmes d'apprentissage par renforcement. Aucune hypothèse n'est faite sur la structure de l'agent. L'interface est compatible avec toute bibliothèque de calcul numérique, comme TensorFlow ou Theano.

La bibliothèque gym est une collection de problèmes de test - environnements - que vous pouvez utiliser pour mettre au point vos algorithmes d'apprentissage par renforcement.

#### Importer un environnement
Tous les [envionnements de Gym](https://gymnasium.farama.org/api/env/) s'instencient et se comportent de la même manière. C'est là tout l'intéret de gym.

##### Observations
Pour prendre des décisions de manière informée, il faut avoir des informations sur l'environnement avec lequel on intéragit. En automatique, on dirait qu'il faut une rétroaction : un système bouclé à de bien meilleures performances qu'un système en boucle ouverte.

La fonction step de l'environnement renvoie exactement ce dont nous avons besoin. Elle renvoie quatre valeurs :

- observation (list) : Le plus souvent une liste, spécifique à l'environnement, représentant votre observation de l'environnement. Par exemple, les données de pixels d'une caméra, les angles et les vitesses des articulations d'un robot, ou l'état du plateau dans un jeu de société.
- récompense (float) : Valeur de la récompense obtenue par l'action précédente. L'échelle varie selon les environnements, mais l'objectif est toujours de cumuler la plus grande la récompense sur un épisode.
- done (booléen) : Indique s'il est temps de réinitialiser l'environnement. La plupart des tâches (mais pas toutes) sont divisées en épisodes bien définis. Si done est égale à 1 alors l'épisode est terminé. (Par exemple, peut-être que le poteau a basculé trop loin, ou que vous avez perdu votre dernière vie).
- info (dict) : informations de diagnostic utiles pour le débogage. L'agent ne doit pas avoir accès à ces informations pour l'apprentissage.
  
Il s'agit d'une implémentation de la classique "boucle agent-environnement". A chaque pas de temps, l'agent choisit une action, et l'environnement renvoie une observation et une récompense.

Chaque épisode débute en appelant reset(), qui renvoie une observation initiale.

##### Espace d'observation et d'action
Chaque environnement est doté d'un espace d'action et d'un espace d'observation. Ces attributs sont de type gym.Space, et ils décrivent le format des actions et des observations valides.

#### Exercice [Taxi](https://gymnasium.farama.org/environments/toy_text/taxi/)

1) Créer l'environnement Taxi (utilisez l'argument render_mode = 'ansi' pour pouvoir afficher l'environnement au fur et a mesure du TD) (1 point)

In [53]:
# Création de l'environnement Taxi avec le rendu ANSI
env = gym.make("Taxi-v3", render_mode='ansi')



2) Utiliser la methode reset pour réinitialiser l'environnement (1 point)

In [54]:
env.reset()

(26, {'prob': 1.0, 'action_mask': array([1, 0, 0, 1, 0, 0], dtype=int8)})

3) Afficher l'environnement (1 point)

In [55]:
# Affichage de  l'environnement
rendered_state = env.render()

# le rendu de l'état
print(rendered_state)

# Fermeture de l'environnement
env.close()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




4) Agent aléatoire. (4 points)

Créer une boucle de 5 étapes, à chaque étape piocher une action aléatoire parmi les actions possibles, appliquer l'action puis afficher le nouvel état de l'environnement et la récompense associée.

In [52]:
for i in range(5):
    action = env.action_space.sample()  
    result = env.step(action) 
    print(f"Resultas de l'Étape {i}: {result}")
    observation, reward, done, info = result[:4]
    print(f"Étape {i+1}: Action: {action}, Observation: {observation}, Récompense: {reward}, Terminé: {done}, Info: {info}")
    env.render()
    if done:
        env.reset()
env.close()


error: display Surface quit

5) Initialisation de la Q-Table (3 points)
   
Créer une matrice numpy de zeros de dimensions (nombre d'état possible , nombre d'action possible)

In [34]:
n_states = env.observation_space.n
n_actions = env.action_space.n
q_table = np.zeros((n_states, n_actions))
print("Dimensions de la Q-table:", q_table.shape)
print("Q-table initialisée avec des zéros:")
print(q_table)
env.close()


Dimensions de la Q-table: (500, 6)
Q-table initialisée avec des zéros:
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


6) Initialisation des hyperparametres

In [35]:
learning_rate = 0.9
discount_rate = 0.8
epsilon = 1.0
decay_rate= 0.005

7) Créer une boucle de 1000 épisodes avec 100 étapes maximum pour chaque épisodes. (6 points)

In [36]:
# pseudo code :

#pour 1000 épisodes:
    #initialiser env
    #pour 99 step:
        #affiche l'environnement
        #en fonction de la valeur d'epsilon soit:
             #choisir une action au hasard (exploration)
        #soit:
            #choisir l'action avec le meilleur score dans la Qtable
        #appliquer l'action en recuperant le nouvel état et la récompense
        #actualiser la Qtable
        #actualiser l'état de l'environnement avec le nouvel état
    #actualiser epsilon

n_states = env.observation_space.n
n_actions = env.action_space.n
q_table = np.zeros((n_states, n_actions))

# pour 1000 épisodes
for episode in range(1000):
    # affiche l'environnement, intialisation
    initial_state_info = env.reset()
    state = initial_state_info[0]

# pour 99 steps
    for step in range(100):
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state, :])

        # appliquer l'action en recuperant le nouvel état et la récompense
        result = env.step(action)

        new_state = result[0]  # Le nouvel état (premier element)
        reward = result[1]
        done = result[2]

        new_state = int(new_state)  # Verifier que c'est un entier

        # Mise à jour de la Q-table
        q_table[state, action] = q_table[state, action] + learning_rate * (
            reward + discount_rate * np.max(q_table[new_state, :]) - q_table[state, action])

        state = new_state  # mise à jour de l'état pour la prochaine action

        if done:
            break

    epsilon = max(epsilon - decay_rate, 0.01)


# Fermeture de l'environnement
env.close()
print("Entraînement terminé.")



Entraînement terminé.


8) Tester l'agent entraîné (4 points)
   
Réinitialiser l'environemment

Créer un boucle permettant de choisir la meilleure action à prendre étant donné l'état de l'environnement en se referant à la Qtable, d'appliquer cette action, la boucle devra se stopper lorsque le taxi reussi sa mission.

In [38]:
# Réinitialiser l'environnement pour commencer un nouvel épisode
state = env.reset()[0]

done = False
total_reward = 0  # Pour garder une trace des récompenses cumulées

while not done:
    # Choisir la meilleure action à partir de la Q-table pour l'état courant
    action = np.argmax(q_table[state, :])

    # Appliquer l'action choisie
    result = env.step(action)
    new_state, reward, done, info = result[:4]

    # Ajouter la récompense à la récompense totale
    total_reward += reward

    # Mettre à jour l'état avec le nouvel état
    state = int(new_state)

    env.render()

    if done:
        print("Le taxi a réussi sa mission!")
        print(f"Récompense totale: {total_reward}")


env.close()

Le taxi a réussi sa mission!
Récompense totale: 5
